In [1]:
!pip install tts

  Using cached TTS-0.22.0.tar.gz (1.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached inflect-7.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached anyascii-0.3.2-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.10.10-cp310-cp310-win_amd64.whl.metadata (7.8 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached pysbd-0.3.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached umap_learn-0.5.6-py3-none-any.whl.metadata (21 kB)
  Using cached pandas-1.5.3-cp310-cp310-win_amd64.whl.metadata (12 kB)
  Using cached matplotlib-3.9.2

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [164 lines of output]
      Copied C:\Users\jigsa\AppData\Local\Temp\pip-install-pu0rp0u_\spacy_b85ed33e7c8c4496aa6c5864ba8edd32\setup.cfg -> C:\Users\jigsa\AppData\Local\Temp\pip-install-pu0rp0u_\spacy_b85ed33e7c8c4496aa6c5864ba8edd32\spacy\tests\package
      Copied C:\Users\jigsa\AppData\Local\Temp\pip-install-pu0rp0u_\spacy_b85ed33e7c8c4496aa6c5864ba8edd32\pyproject.toml -> C:\Users\jigsa\AppData\Local\Temp\pip-install-pu0rp0u_\spacy_b85ed33e7c8c4496aa6c5864ba8edd32\spacy\tests\package
      Cythonizing sources
      Compiling spacy/training/example.pyx because it changed.
      Compiling spacy/parts_of_speech.pyx because it changed.
      Compiling spacy/strings.pyx because it changed.
      Compiling spacy/lexeme.pyx because it changed.
      Compiling spacy/vocab.pyx because it changed.
      Compiling spacy/attrs.pyx because it changed.
      Compili

In [24]:
%pip install tts

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.0.10.2 Requires-Python >=3.6.0, <3.9; 0.0.10.3 Requires-Python >=3.6.0, <3.9; 0.0.11 Requires-Python >=3.6.0, <3.9; 0.0.12 Requires-Python >=3.6.0, <3.9; 0.0.13.1 Requires-Python >=3.6.0, <3.9; 0.0.13.2 Requires-Python >=3.6.0, <3.9; 0.0.14.1 Requires-Python >=3.6.0, <3.9; 0.0.15 Requires-Python >=3.6.0, <3.9; 0.0.15.1 Requires-Python >=3.6.0, <3.9; 0.0.9 Requires-Python >=3.6.0, <3.9; 0.0.9.1 Requires-Python >=3.6.0, <3.9; 0.0.9.2 Requires-Python >=3.6.0, <3.9; 0.0.9a10 Requires-Python >=3.6.0, <3.9; 0.0.9a9 Requires-Python >=3.6.0, <3.9; 0.1.0 Requires-Python >=3.6.0, <3.10; 0.1.1 Requires-Python >=3.6.0, <3.10; 0.1.2 Requires-Python >=3.6.0, <3.10; 0.1.3 Requires-Python >=3.6.0, <3.10; 0.10.0 Requires-Python >=3.7.0, <3.11; 0.10.1 Requires-Python >=3.7.0, <3.11; 0.10.2 Requires-Python >=3.7.0, <3.11; 0.11.0 Requires-Python >=3.7.0, <3.11; 0.11.1 Requires-Python >=3.7.0, <3.11; 0.12.0 Requires-Python >=3

In [ ]:
# This code started as a direct copy from opencv
# https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

#pyttsx3 courtesy of: https://pypi.org/project/pyttsx3/

import numpy as np
import cv2 as cv
from ultralytics import YOLO
from TTS.api import TTS  # Import Coqui TTS

# If DEBUG is True we will output all the detected signs and their confidence
DEBUG = True

# Initialize TTS engine (Coqui TTS)
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", gpu=False)

# CoquiTTS doesnt support gender selection like pyttsx3)
def select_voice(gender='m'):
    print(f"Selected {gender.upper()} voice, but gender-specific voices depend on the model for Coqui TTS.")

# Prompt to select gender
gender = input("Select voice (m: male / f: female) : ").strip().lower()
select_voice(gender)

# Load the YOLOv8 model, this is loading our custom trained weights for our model.
model = YOLO("best_v3_yolov11.pt")

cap = cv.VideoCapture(0)

# Stored captured text
captured_text = []

# Stores captured confidence
captured_confidence = []

# Set a threshold for the sign to register
confidence_requirement = 0.90
#confidence_requirement = 0.30

# Counts the number of consecutive significant signs
count = 0

# Counts the number of consecutive insignificant signs
noise_count = 0

# Keeps track of the last sign
last = None

translator = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","DEL","NOTHING"," "]
live_text = "Text: "

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # Run YOLOv8 inference on the frame
    results = model(frame, verbose=False)
    top_class = results[0].probs.top1
    top_confidence = results[0].probs.top1conf  # Get confidence of the top-class prediction
    
    # If the confidence of the sign is above the threshold 
    if top_confidence >= confidence_requirement:
        if top_class == last:
            count += 1
        else:
            count = 1
        
        # If there are 3 consecutive significant signs track it
        if count == 3:
            sign = translator[top_class]

            if DEBUG == True:
                captured_text.append(sign)
                captured_confidence.append(top_confidence)

            if sign == "SPACE":
                captured_text.append("_")  # Store "_" for space in captured_text
                live_text += "_"  # Display "_" instead of " " in live_text
            elif sign == "DEL":
                live_text = live_text[:-1]  # Removes the last character
            elif sign != "NOTHING":
                live_text += sign  # Appends the recognized gesture to live_text

        last = top_class
        noise_count = 0 
    else:
        noise_count += 1

    if noise_count == 3:
        count = 0
        noise_count = 0
    
    # Visualize the results on the frame
    cv.putText(frame, live_text, (10, 460), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_4)
    cv.imshow("Capture", frame)
    
    if cv.waitKey(1) == ord('q'):
        if DEBUG == True:
            print("\n\nCaptured Text:")
            for i in range(len(captured_text)):
                print("Translated text: ", captured_text[i], " Confidence: ", captured_confidence[i].item())
        
        spoken_text = ''.join(captured_text).replace("_", " ").strip()
        print("Speaking: ", spoken_text)
        tts.tts_to_file(text=spoken_text, file_path="output.wav")
        print("Playing audio...")
        # Playing the saved file
        from playsound import playsound
        playsound("output.wav")

        break

cap.release()
cv.destroyAllWindows()

# a function to replay the captured text after ending the webcam feed
while True:
    replay_choice = input("Press 'R' to replay the sentence or 'E' to exit: ").strip().lower()
    if replay_choice == 'r':
        print("Replaying: ", spoken_text)
        playsound("output.wav")
    elif replay_choice == 'e':
        print("Exiting.")
        break

ModuleNotFoundError: No module named 'TTS'

In [5]:
#Courtesy of: https://www.geeksforgeeks.org/essential-opencv-functions-to-get-started-into-computer-vision/

# Mapping each letter+space to the corresponding ASL image path
asl_image_paths = {
    'A': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\A.jpg',
    'B': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\B.jpg',
    'C': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\C.jpg',
    'D': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\D.jpg',
    'E': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\E.jpg',
    'F': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\F.jpg',
    'G': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\G.jpg',
    'H': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\H.jpg',
    'I': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\I.jpg',
    'J': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\J.jpg',
    'K': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\K.jpg',
    'L': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\L.jpg',
    'M': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\M.jpg',
    'N': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\N.jpg',
    'O': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\O.jpg',
    'P': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\P.jpg',
    'Q': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\Q.jpg',
    'R': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\R.jpg',
    'S': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\S.jpg',
    'T': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\T.jpg',
    'U': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\U.jpg',
    'V': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\V.jpg',
    'W': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\W.jpg',
    'X': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\X.jpg',
    'Y': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\Y.jpg',
    'Z': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\Z.jpg',
    '_': r'C:\Users\jigsa\Downloads\ASL TTS Dataset\SPACE.jpg'
}

# a dictionary to store loaded and resized images to avoid loading each time
asl_images = {}

# pre-load and resize all images to 500x500
for letter, path in asl_image_paths.items():
    img = cv.imread(path)
    if img is not None:
        resized_img = cv.resize(img, (500, 500))
        asl_images[letter] = resized_img
    else:
        print(f"The image for '{letter}' is not found at : {path}")

# a function to display each letter as an ASL gesture
def display_asl_gesture(text):
    for char in text:
        # again, convert the space (' ') to '_' for the dictionary
        char = '_' if char == ' ' else char.upper()

        # get the pre-loaded and resized ASL image
        img = asl_images.get(char)
        if img is not None:
            cv.imshow(f"ASL Gesture for {char}", img)
            cv.waitKey(1000)  #set the display time for each image to 1 second
            cv.destroyWindow(f"ASL Gesture for {char}")
        else:
            print(f"ASL gesture for '{char}' is not available.")

# input text to be translated to ASL
input_text = input("Enter text to translate to ASL gestures: ").strip()

# call the display ASL function
display_asl_gesture(input_text)

cv.destroyAllWindows()